<a href="https://colab.research.google.com/github/gillesretiere/langdeck/blob/novomaster/langdeck_pipe_2024_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata
import ast

!pip3 install pyclean

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

# python .env see -> https://saurabh-kumar.com/python-dotenv/
!pip install python-dotenv
from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()
dotenv_config = dotenv_values(".env")
app_path = dotenv_config["APP_PATH"]

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)

#====== Airtable ================
!pip install airtable-python-wrapper
from airtable import Airtable

#====== Deep Translate ================
!pip install -U deep-translator
!pip install --upgrade deepl

#====== Dictionaries ================
# larousse API
!pip install larousse-api-sunbro

#====== Lemmatizers ================
# https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

#====== Text To Speech
# about quotas : https://stackoverflow.com/questions/65980562/gtts-tts-gttserror-429-too-many-requests-from-tts-api-probable-cause-unknow
import os
!pip install gtts --upgrade
from gtts import gTTS

# CDN Cloudinary for upload
!pip3 install cloudinary
import cloudinary
import cloudinary.uploader
import cloudinary.api
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url

!export PYTHONIOENCODING=utf8

%load_ext autoreload
%autoreload 2
%cd /content/drive/MyDrive/langdeck/
!pyclean .
import sys, os
sys.path.append('/content/drive/MyDrive/langdeck/py_modules')
import py_modules
py_modules.test()

# .env
from dotenv import load_dotenv
from dotenv import dotenv_values
%cd /content/drive/MyDrive/langdeck/
load_dotenv()
dotenv_config = dotenv_values(".env")

# Config CDN
cloudinary.config(
  cloud_name = dotenv_config["CLOUDINARY_CLOUD_NAME"],
  api_key = dotenv_config["CLOUDINARY_API_KEY"],
  api_secret = dotenv_config["CLOUDINARY_API_SECRET"],
  secure = True
)

load_dotenv()
dotenv_config = dotenv_values(".env")

print ("Environnement prêt.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 783.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 277.3 kB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.3.1
Uninstalling gspread-dataframe-3.3.1:
  Successfully uninstalled gspread-dataframe-3.3.1
Mounted at /content/drive
/content/drive/MyDrive/langdeck
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=4eebe6fd2bf3f2bedda3ee1494820ee57535de55428119a17c4a930a0eddb90b
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /tmp/pip-req-

# Cnx Google Sheet DB
Ici on teste la cnx avec la base d'intégration qui est une table GSHEET

In [2]:
db_lgdk_intgr = dotenv_config["DB_LGDK_INTGR"]
#--- integration database
sheet = db_lgdk_intgr
wb_intgr  = gc.open_by_url(sheet)
#--- Liste des codes ISO639-3
df_lang_alpha3 = py_modules.load_df_from_gsheet (wb_intgr, "ref_iso-639-3")
#--- Liste des langues (issues d'un Corpus)
df_lang_corpus = py_modules.load_df_from_gsheet (wb_intgr, "ref_lang_corpus")
# df_lang_corpus.head()

# Cnx Airtable DB
Ici on teste la cnx avec la base de DESIGN qui est une table Airtable
##### doc : https://airtable-python-wrapper.readthedocs.io/en/airtable-python-wrapper/index.html


In [3]:
#---- Base Airtable 2023
api_key_airtable = dotenv_config["AIRTABLE_PIPE23_API_KEY"]
headers = {"Authorization": "Bearer " + api_key_airtable,"Content-Type" : "application/json"}
base_id = dotenv_config["AIRTABLE_PIPE23_BASE_ID"]
table_name = dotenv_config["AIRTABLE_PIPE23_TBL_NAME"]
airtable = Airtable(base_id, table_name, api_key_airtable)
active_filter = "{Status}='Done'"

# Langues

In [4]:
tbd_languages = Airtable(base_id, "languages", api_key_airtable)
lang_filter = "{lang_is_available}='TRUE'"
vw_lang_grid = tbd_languages.get_all(
    view='view_grid',
    sort=['lang_id','lang_is_available'],
    formula=lang_filter,
    )

# Thèmes

## Intégration

In [5]:
tbd_themes = Airtable(base_id, "themes", api_key_airtable)
vw_themes_grid = tbd_themes.get_all(
    view='view_grid',
    sort=['theme_name'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_themes = pd.DataFrame.from_records((r['fields'] for r in vw_themes_grid))
df_themes = df_themes[["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "lessons",
                 "stories",
                 "theme_illustration",
                 ]]


#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
py_modules.save_df_to_gsheet (wb_intgr, "tbl_languages", df_languages)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_themes = pd.concat([df_themes, df_t.reindex(df_themes.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_themes,
        id_vars=["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "lessons",
                 "stories",
                 "theme_illustration",
                 ],
        var_name="theme_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["theme_translation_id"] = df_w_temp[['theme_rec_id', 'theme_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)


#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_themes = py_modules.load_df_from_gsheet (wb_intgr, "tbl_themes")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_themes[["theme_translation_id",
                  "theme_translation",
                  "theme_translation_status",
                  "theme_audio",
                  "theme_audio_url",
                  "theme_audio_url_fr"]],
    on="theme_translation_id",
    how="left")

## Traduction

In [7]:
df_w_tr["theme_translation"] = df_w_tr["theme_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["theme_translation_status"] = df_w_tr["theme_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["theme_translation","theme_translation_status"]] = df_w_tr[["theme_translation",
                                                                   "theme_name",
                                                                   "theme_language",
                                                                   "theme_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["theme_audio"] = df_w_tr[["theme_translation_id","theme_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [8]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["theme_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["theme_translation","theme_language","theme_audio","theme_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['theme_audio_url'] = df_tts['theme_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["theme_audio_url"] = df_tts[["theme_language","theme_audio","theme_audio_url","theme_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["theme_audio_url_fr"] = df_tts["theme_rec_id"].map(
    lambda x:py_modules.theme_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [9]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_themes", df_tts)

# Lessons

## Intégration

In [10]:
tbd_lessons = Airtable(base_id, "lessons", api_key_airtable)
vw_lessons_grid = tbd_lessons.get_all(
    view='view_grid',
    sort=['lesson'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
vk_col_lessons = ["lesson_rec_id",
      "lesson",
      "lesson_story",
      "lesson_pictorial",
      "lesson_quiz",
      "lesson_number",
      "lesson_illustration",
      "lesson_related_theme",
      "Status",
      ]
df_lessons = pd.DataFrame.from_records((r['fields'] for r in vw_lessons_grid))
df_lessons = df_lessons[vk_col_lessons]

In [11]:
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_lessons = pd.concat([df_lessons, df_t.reindex(df_lessons.index)], axis=1)

#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_lessons,
        id_vars=vk_col_lessons,
        var_name="lesson_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["lesson_translation_id"] = df_w_temp[['lesson_rec_id', 'lesson_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
vk_col_t_lessons = [
    "lesson_translation_id",
    "lesson_translation",
    "lesson_translation_status",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr"
    ]
df_tbl_lessons = py_modules.load_df_from_gsheet (wb_intgr, "tbl_lessons")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_lessons[vk_col_t_lessons],
    on="lesson_translation_id",
    how="left")


## Traduction

In [12]:
df_w_tr["lesson_translation"] = df_w_tr["lesson_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["lesson_translation_status"] = df_w_tr["lesson_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["lesson_translation","lesson_translation_status"]] = df_w_tr[["lesson_translation",
                                                                   "lesson",
                                                                   "lesson_language",
                                                                   "lesson_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["lesson_audio"] = df_w_tr[["lesson_translation_id","lesson_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [13]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["lesson_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["lesson_translation","lesson_language","lesson_audio","lesson_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['lesson_audio_url'] = df_tts['lesson_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
df_tts["lesson_audio_url"] = df_tts[["lesson_language","lesson_audio","lesson_audio_url","lesson_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["lesson_audio_url_fr"] = df_tts["lesson_rec_id"].map(
    lambda x:py_modules.table_get_audio_url_fr(x,
                                    df_tts,
                                    "lesson_translation_id",
                                    "lesson_audio_url",
                                    ))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [14]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_lessons", df_tts)

# Stories

## Intégration

In [15]:
tbd_stories = Airtable(base_id, "stories", api_key_airtable)
vw_stories_grid = tbd_stories.get_all(
    view='view_grid',
    sort=['story_name','story_level'],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_stories = pd.DataFrame.from_records((r['fields'] for r in vw_stories_grid))
df_stories.rename(columns={'story_theme': 'story_theme_rec_id'}, inplace=True)
df_stories["story_theme"] = df_stories["story_theme_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_themes, x, "theme_name"),
    na_action="ignore"
)
vk_col_stories = ["story_rec_id",
                 "story_name",
                 "story_episode",
                 "story_level",
                 "story_theme",
                 "story_illustration",
                 "paragraphs",
                 "story_desc",
                 "story_desc_tr_status",
                 "story_synopsis",
                 "story_synopsis_tr_status",
                 "story_lesson_header",
                 "story_related_lesson",
                 ]
df_stories = df_stories[vk_col_stories]
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_stories = pd.concat([df_stories, df_t.reindex(df_stories.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_stories,
        id_vars=vk_col_stories,
        var_name="story_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["story_translation_id"] = df_w_temp[['story_rec_id', 'story_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_stories[["story_translation_id",
                  "story_translation",
                  "story_translation_status",
                  "story_audio",
                  "story_audio_url",
                  "story_audio_url_fr",
                  "story_desc_translation",
                  "story_synopsis_translation",
                  ]],
    on="story_translation_id",
    how="left")

## Traduction

In [16]:
df_w_tr["story_translation"] = df_w_tr["story_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["story_translation_status"] = df_w_tr["story_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["story_translation","story_translation_status"]] = df_w_tr[["story_translation",
                                                                   "story_name",
                                                                   "story_language",
                                                                   "story_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["story_audio"] = df_w_tr[["story_translation_id","story_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [17]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["story_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["story_translation","story_language","story_audio","story_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['story_audio_url'] = df_tts['story_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["story_audio_url"] = df_tts[["story_language","story_audio","story_audio_url","story_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["story_audio_url_fr"] = df_tts["story_rec_id"].map(
    lambda x:py_modules.story_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


### Translation of story description

In [18]:
# translate if necessary # status == NaN OR START
df_tts["story_synopsis_translation"] = df_tts["story_synopsis_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_tts[["story_synopsis_translation","story_synopsis_tr_status"]] = df_tts[
    ["story_synopsis_translation",
     "story_synopsis",
     "story_language",
     "story_synopsis_tr_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# translate if necessary # status == NaN OR START
df_tts["story_desc_translation"] = df_tts["story_desc_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

df_tts[["story_desc_translation","story_desc_tr_status"]] = df_tts[
    ["story_desc_translation",
     "story_desc",
     "story_language",
     "story_desc_tr_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")

In [19]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_stories", df_tts)

# Words

In [ ]:
tbd_words = Airtable(base_id, "words", api_key_airtable)
vw_words_grid = tbd_words.get_all(
    view='view_grid',
    sort=['word','word_type'],
    formula=active_filter,
    )

## Intégration

In [ ]:
#=======================================================
# Transfo en dataframe
#=======================================================
df_words = pd.DataFrame.from_records((r['fields'] for r in vw_words_grid))
# on ne garde que les colonnes utiles
df_words = df_words[["word_rec_id",
                     "word",
                     "word_type",
                     "word_lemmas",
                     "word_notes",
                     "word_nutri_notes",
                     "word_notes_status",
                     "word_nutri_notes_status",
                     ]]
# Capitalisation
df_words["word"]=df_words["word"].map(lambda x:x[:1].upper() + x[1:])
# on convertit les rec_id en valeur litérale
df_words.rename(columns={'word_lemmas': 'word_lemmas_rec_id'}, inplace=True)
# on convertit les rec_id en valeur litérale
df_words["word_lemmas"] = df_words["word_lemmas_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")

#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_words = pd.concat([df_words, df_t.reindex(df_words.index)], axis=1)

#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_words,
        id_vars=["word_rec_id",
                 "word",
                 "word_type",
                 "word_lemmas",
                 "word_lemmas_rec_id",
                 "word_notes",
                 "word_nutri_notes",
                 "word_notes_status",
                 "word_nutri_notes_status",
                 ],
        var_name="word_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["word_translation_id"] = df_w_temp[['word_rec_id', 'word_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_words[["word_translation_id",
                  "word_translation",
                  "word_translation_status",
                  "word_audio",
                  "word_audio_url",
                  "word_audio_url_fr",
                  "word_notes_translation",
                  "word_notes_translation_status",
                  "word_nutri_notes_translation",
                  "word_nutri_notes_translation_status",
                  ]],
    on="word_translation_id",
    how="left")


## Traduction

In [ ]:
df_w_tr["word_translation"] = df_w_tr["word_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_translation_status"] = df_w_tr["word_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# il faut forcer la maj car les lemmes ont peut-être changé => on recrée la colonne from scratch
df_w_tr["word_lemmas_tr_rec_id"]=df_w_tr[["word_lemmas_rec_id","word_language"]].apply(
    lambda x:py_modules.f_word_lemmas_tr_rec_id(x[0],x[1]) if type(x[0])==list else np.nan,
    axis=1)
# translate if necessary # status == NaN OR START
df_w_tr[["word_translation","word_translation_status"]] = df_w_tr[["word_translation",
                                                                   "word",
                                                                   "word_language",
                                                                   "word_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# on convertit les rec_id en valeur litérale
df_w_tr["word_lemmas_tr"] = df_w_tr["word_lemmas_tr_rec_id"].map(
    lambda x:py_modules.f_word_lemmas_tr(x, df_w_tr),
    na_action="ignore")

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["word_audio"] = df_w_tr[["word_translation_id","word_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)



Translated Consultati... from fre to nl >>> Overleg...
Translated Consultati... from fre to en >>> Consultati...
Translated Consultati... from fre to fr >>> Consultati...
Translated Consultati... from fre to de >>> Beratung...


## Notes and Nutri notes

In [ ]:
df_w_tr["word_notes_translation"] = df_w_tr["word_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_notes_translation_status"] = df_w_tr["word_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [ ]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_notes_translation","word_notes_translation_status"]] = df_w_tr[
    ["word_notes_translation",
     "word_notes",
     "word_language",
     "word_notes_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [ ]:
df_w_tr["word_nutri_notes_translation"] = df_w_tr["word_nutri_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_nutri_notes_translation_status"] = df_w_tr["word_nutri_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [ ]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_nutri_notes_translation","word_nutri_notes_translation_status"]] = df_w_tr[
    ["word_nutri_notes_translation",
     "word_nutri_notes",
     "word_language",
     "word_nutri_notes_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["word_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["word_translation","word_language","word_audio","word_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['word_audio_url'] = df_tts['word_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["word_audio_url"] = df_tts[["word_language","word_audio","word_audio_url","word_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["word_audio_url_fr"] = df_tts["word_rec_id"].map(
    lambda x:py_modules.word_get_audio_url_fr(x, df_tts))

py_modules.save_df_to_gsheet (wb_intgr, "tbl_words", df_tts)

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger
/content/drive/MyDrive/langdeck/assets/audio/ai/dut/recRUrPp0aC5oNvVc-dut.mp3(nl) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/eng/recRUrPp0aC5oNvVc-eng.mp3(en) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/fre/recRUrPp0aC5oNvVc-fre.mp3(fr) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/ger/recRUrPp0aC5oNvVc-ger.mp3(de) : okay
iso3:dut ; local_folder:/content/drive/MyDrive/langdeck/assets/audio/ai/dut/ ; path:recRUrPp0aC5oNvVc-dut.mp3
http://res.cloudinary.com/dhc7ovnwk/video/upload/v1704293003/langdeck/assets/audio/ai/dut/recRUrPp0aC5oNvVc-dut.mp3
iso3:eng ; local_folder:/content/drive/MyDrive/langdeck/assets/audio/ai/eng/ ; path:recRUrPp0aC5oNvVc-eng.mp3
http://res.cloudinary.com/dhc7ovnwk/video/upload/v1704293004/langdeck/assets/audio/ai/eng/recRUrPp0a

# Phrases

## Intégration

In [ ]:
tbd_phrases = Airtable(base_id, "phrases", api_key_airtable)
vw_phrases_grid = tbd_phrases.get_all(
    view='view_grid',
    sort=['phrase_related_story','phrase_paragraph','phrase_position'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_phrases = pd.DataFrame.from_records((r['fields'] for r in vw_phrases_grid))
df_phrases.rename(columns={'phrase_words': 'phrase_words_rec_id'}, inplace=True)
df_phrases["phrase_words"] = df_phrases["phrase_words_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")
# récupérer la saynète à partir de son id
df_phrases.rename(columns={'phrase_related_story': 'phrase_related_story_rec_id'}, inplace=True)
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_stories, x, "story_name"),
    na_action="ignore")

df_phrases["phrase_html"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html(x[0], x[1]),
    axis=1)

df_phrases["phrase_html"] = df_phrases[["phrase_html","phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_html_rec_id","phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id_kw(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_kw"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

In [ ]:
vk_col_phrases = [
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_status",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_status",
    "phrase_notes_2_status",
    ]
df_phrases = df_phrases[vk_col_phrases]

#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_phrases = pd.concat([df_phrases, df_t.reindex(df_phrases.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_phr_temp = pd.melt(df_phrases,
        id_vars = vk_col_phrases,
        var_name = "phrase_language",
        value_name = "translation")
# hash incl. language iso3
df_phr_temp["phrase_translation_id"] = df_phr_temp[['phrase_rec_id', 'phrase_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_phr_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_phrases = py_modules.load_df_from_gsheet (wb_intgr, "tbl_phrases")
df_phr_tr = pd.merge(
    df_phr_temp,
    df_tbl_phrases[[
        "phrase_translation_id",
        "phrase_translation",
        "phrase_translation_status",
        "phrase_audio",
        "phrase_audio_url",
        "phrase_audio_url_fr",
        "phrase_notes_translation",
        "phrase_notes_translation_status",
        "phrase_notes_2_translation",
        "phrase_notes_2_translation_status",
        ]],
    on="phrase_translation_id",
    how="left")

In [ ]:
df_phr_tr["phrase_notes_translation"] = df_phr_tr["phrase_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_translation_status"] = df_phr_tr["phrase_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation"] = df_phr_tr["phrase_notes_2_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation_status"] = df_phr_tr["phrase_notes_2_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

## Traduction

### Traductions : mots à ne pas traduire
En attendant d'avoir les fonctionnalités de DeepL, on ignore les tags
```
def phrase_notes_markdown_to_html_kw (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<span translate="no">', temp)
  target = re.sub(r'\`\*+', '</span>', temp)
  return target
  
def phrase_notes_markdown_to_html_kw_2 (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<code>', temp)
  target = re.sub(r'\`\*+', '</code>', temp)
  return target
```



In [ ]:
df_phr_tr["phrase_notes"] = df_phr_tr["phrase_notes"].map(lambda x:py_modules.phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)
df_phr_tr["phrase_notes_2"] = df_phr_tr["phrase_notes_2"].map(lambda x:py_modules.phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)

In [ ]:
# notes
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_notes_translation","phrase_notes_translation_status"]] = df_phr_tr[
    ["phrase_notes_translation",
     "phrase_notes",
     "phrase_language",
     "phrase_notes_translation_status",
     "phrase_notes_status",
     ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='Start' or x[4]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [ ]:
# notes
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_notes_2_translation","phrase_notes_2_translation_status"]] = df_phr_tr[
    ["phrase_notes_2_translation",
     "phrase_notes_2",
     "phrase_language",
     "phrase_notes_2_translation_status",
     "phrase_notes_2_status",
     ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='Start' or x[4]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [ ]:
df_phr_tr["phrase_translation"] = df_phr_tr["phrase_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_translation_status"] = df_phr_tr["phrase_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_translation","phrase_translation_status"]] = df_phr_tr[[
    "phrase_translation",
    "phrase",
    "phrase_language",
    "phrase_translation_status",
    "phrase_status",
    ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3]=='Start' or x[4]=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_phr_tr["phrase_audio"] = df_phr_tr[["phrase_translation_id","phrase_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x.upper()=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_phr_tr.loc[df_phr_tr["phrase_language"].isin(vk_tts)]
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    x[1] if x[1].lower() == "update" else x[0],
    axis=1)
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[[
    "phrase_translation",
    "phrase_language",
    "phrase_audio",
    "phrase_translation_status",
    ]].apply(
    lambda x:py_modules.text2speech(
      x[0],
      df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
      audio_path + x[1] + "/" + x[2].split("/")[-1],
      False
    )
    if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
      or x[3].lower()=="update"
    else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['phrase_audio_url'] = df_tts['phrase_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["phrase_audio_url"] = df_tts[[
    "phrase_language",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_translation_status",
    ]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2])
      or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["phrase_audio_url_fr"] = df_tts["phrase_rec_id"].map(
    lambda x:py_modules.phrase_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [ ]:
#=======================================================
# TRADUCTION DES MOTS ASSOCIES
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
# # construit un id concaténé avec le code langue pour chaque element du vecteur
df_tts["phrase_words_tr_rec_id"] = df_tts[["phrase_words_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)
# récupère le mot traduit depuis son identifiant
df_tts["phrase_words_tr"] = df_tts["phrase_words_tr_rec_id"].map(
    lambda x:py_modules.set_phrase_words_tr(x, df_tbl_words),
    na_action='ignore')
#=======================================================
# TRADUCTION DES META DONNEES (STORY, THEME, ETC.)
#=======================================================
df_tts["phrase_related_story_tr_rec_id"] = df_tts[["phrase_related_story_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)

# récupère le mot traduit depuis son identifiant
df_tts["phrase_related_story_tr"] = df_tts["phrase_related_story_tr_rec_id"].map(
    lambda x:py_modules.get_story_tr(x, df_tbl_stories),
    na_action='ignore')

In [ ]:
# on rebascule les états si il y a eu update
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    "Pending" if x[1].lower() == "update" else x[0],
    axis=1)

py_modules.save_df_to_gsheet (wb_intgr, "tbl_phrases", df_tts)

# Supplement (TPs)

## Intégration

In [ ]:
tbd_tps = Airtable(base_id, "tps", api_key_airtable)
vw_tps_grid = tbd_tps.get_all(
    view='view_grid',
    sort=['tp_name',],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_tps = pd.DataFrame.from_records((r['fields'] for r in vw_tps_grid))
df_tps.rename(columns={'tp_rel_story': 'tp_rel_story_rec_id'}, inplace=True)
df_tps["tp_rel_story"] = df_tps["tp_rel_story_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_stories, x, "story_name"),
    na_action="ignore"
)

vk_col_tps = ["tp_rec_id",
              "tp_name",
              "tp_rel_story_rec_id",
              "tp_rel_story",
              "tp_illustration",
              "tp_summary",
              "tp_lesson_header",
              ]
df_tps = df_tps[vk_col_tps]

#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_tps = pd.concat([df_tps, df_t.reindex(df_tps.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_tps,
        id_vars=vk_col_tps,
        var_name="tp_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["tp_translation_id"] = df_w_temp[['tp_rec_id', 'tp_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_tps = py_modules.load_df_from_gsheet (wb_intgr, "tbl_tps")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_tps[["tp_translation_id",
                  "tp_translation",
                  "tp_translation_status",
                  "tp_audio",
                  "tp_audio_url",
                  "tp_audio_url_fr",
                  "tp_summary_translation",
                  "tp_summary_translation_status",
                  ]],
    on="tp_translation_id",
    how="left")



## Traduction

In [ ]:
df_w_tr["tp_translation"] = df_w_tr["tp_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["tp_translation_status"] = df_w_tr["tp_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["tp_translation","tp_translation_status"]] = df_w_tr[["tp_translation",
                                                                   "tp_name",
                                                                   "tp_language",
                                                                   "tp_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["tp_audio"] = df_w_tr[["tp_translation_id","tp_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

In [ ]:
df_w_tr["tp_summary_translation"] = df_w_tr["tp_summary_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["tp_summary_translation_status"] = df_w_tr["tp_summary_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["tp_summary_translation","tp_summary_translation_status"]] = df_w_tr[["tp_summary_translation",
                                                                   "tp_summary",
                                                                   "tp_language",
                                                                   "tp_summary_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

## TTS

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["tp_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["tp_translation","tp_language","tp_audio","tp_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['tp_audio_url'] = df_tts['tp_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
df_tts["tp_audio_url"] = df_tts[["tp_language","tp_audio","tp_audio_url","tp_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["tp_audio_url_fr"] = df_tts["tp_rec_id"].map(
    lambda x:py_modules.table_get_audio_url_fr(x,
                                    df_tts,
                                    "tp_translation_id",
                                    "tp_audio_url",
                                    ))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [ ]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_tps", df_tts)

# TP Phrases

## Intégration

In [ ]:
tbd_tp_phrases = Airtable(base_id, "tp_phrases", api_key_airtable)
vw_tp_phrases_grid = tbd_tp_phrases.get_all(
    view='view_grid',
    sort=['tp_phrase_rel_tp','tp_phrase_order',],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_tp_phrases = pd.DataFrame.from_records((r['fields'] for r in vw_tp_phrases_grid))
df_tp_phrases.rename(columns={'tp_phrase_words': 'tp_phrase_words_rec_id'}, inplace=True)
df_tp_phrases["tp_phrase_words"] = df_tp_phrases["tp_phrase_words_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")
# récupérer la saynète à partir de son id
df_tp_phrases.rename(columns={'tp_phrase_rel_tp': 'tp_phrase_rel_tp_rec_id'}, inplace=True)
df_tp_phrases["tp_phrase_rel_tp"] = df_tp_phrases["tp_phrase_rel_tp_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_tps, x, "tp_name"),
    na_action="ignore")

df_tp_phrases["tp_phrase_html"] = df_tp_phrases[["tp_phrase_rtf","tp_phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html(x[0], x[1]),
    axis=1)

df_tp_phrases["tp_phrase_html"] = df_tp_phrases[["tp_phrase_html","tp_phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_tp_phrases["tp_phrase_html_rec_id"] = df_tp_phrases[["tp_phrase_rtf","tp_phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

df_tp_phrases["tp_phrase_html_rec_id"] = df_tp_phrases[["tp_phrase_html_rec_id","tp_phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id_kw(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_tp_phrases["tp_phrase_html_kw"] = df_tp_phrases[["tp_phrase_rtf","tp_phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

## Traduction

In [ ]:
vk_col_tp_phrases = [
    "tp_phrase_rec_id",
    "tp_phrase",
    "tp_phrase_words_rec_id",
    "tp_phrase_words",
    "tp_phrase_html",
    "tp_phrase_html_rec_id",
    "tp_phrase_html_kw",
    "tp_phrase_rel_tp",
    "tp_phrase_rel_tp_rec_id",
    "tp_phrase_order",
    "tp_phrase_status",
    ]
df_tp_phrases = df_tp_phrases[vk_col_tp_phrases]
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_tp_phrases = pd.concat([df_tp_phrases, df_t.reindex(df_tp_phrases.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_phr_temp = pd.melt(df_tp_phrases,
        id_vars = vk_col_tp_phrases,
        var_name = "tp_phrase_language",
        value_name = "translation")
# hash incl. language iso3
df_phr_temp["tp_phrase_translation_id"] = df_phr_temp[['tp_phrase_rec_id', 'tp_phrase_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_phr_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_tp_phrases = py_modules.load_df_from_gsheet (wb_intgr, "tbl_tp_phrases")
df_phr_tr = pd.merge(
    df_phr_temp,
    df_tbl_tp_phrases[[
        "tp_phrase_translation_id",
        "tp_phrase_translation",
        "tp_phrase_translation_status",
        "tp_phrase_audio",
        "tp_phrase_audio_url",
        "tp_phrase_audio_url_fr",
        ]],
    on="tp_phrase_translation_id",
    how="left")

In [ ]:
df_phr_tr["tp_phrase_translation"] = df_phr_tr["tp_phrase_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["tp_phrase_translation_status"] = df_phr_tr["tp_phrase_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_phr_tr[["tp_phrase_translation","tp_phrase_translation_status"]] = df_phr_tr[[
    "tp_phrase_translation",
    "tp_phrase",
    "tp_phrase_language",
    "tp_phrase_translation_status",
    "tp_phrase_status",
    ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3]=='Start' or x[4]=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_phr_tr["tp_phrase_audio"] = df_phr_tr[["tp_phrase_translation_id","tp_phrase_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

Translated Quel est v... from fre to nl >>> Wat is de ...
Translated Veuillez p... from fre to nl >>> Specificee...
Translated Où avez-vo... from fre to nl >>> Waar doet ...
Translated Quel est v... from fre to en >>> What is yo...
Translated Veuillez p... from fre to en >>> Please spe...
Translated Où avez-vo... from fre to en >>> Where does...
Translated Quel est v... from fre to fr >>> Quel est v...
Translated Veuillez p... from fre to fr >>> Veuillez p...
Translated Où avez-vo... from fre to fr >>> Où avez-vo...
Translated Quel est v... from fre to de >>> Was ist Ih...
Translated Veuillez p... from fre to de >>> Bitte gebe...
Translated Où avez-vo... from fre to de >>> Wo tut es ...


## TTS

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x.upper()=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()

df_tts = df_phr_tr.loc[df_phr_tr["tp_phrase_language"].isin(vk_tts)]
df_tts["tp_phrase_translation_status"] = df_tts[["tp_phrase_translation_status","tp_phrase_status"]].apply(
    lambda x:
    x[1] if x[1].lower() == "update" else x[0],
    axis=1)
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[[
    "tp_phrase_translation",
    "tp_phrase_language",
    "tp_phrase_audio",
    "tp_phrase_translation_status",
    ]].apply(
    lambda x:py_modules.text2speech(
      x[0],
      df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
      audio_path + x[1] + "/" + x[2].split("/")[-1],
      False
    )
    if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
      or x[3].lower()=="update"
    else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['tp_phrase_audio_url'] = df_tts['tp_phrase_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["tp_phrase_audio_url"] = df_tts[[
    "tp_phrase_language",
    "tp_phrase_audio",
    "tp_phrase_audio_url",
    "tp_phrase_translation_status",
    ]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2])
      or x[3].lower() == "update"
    else x[2],
    axis=1)

df_tts["tp_phrase_audio_url_fr"] = df_tts["tp_phrase_rec_id"].map(
    lambda x:py_modules.table_get_audio_url_fr(x,
                                    df_tts,
                                    "tp_phrase_translation_id",
                                    "tp_phrase_audio_url",
                                    ))
#=======================================================
# TRADUCTION DES MOTS ASSOCIES
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
# # construit un id concaténé avec le code langue pour chaque element du vecteur
df_tts["tp_phrase_words_tr_rec_id"] = df_tts[["tp_phrase_words_rec_id","tp_phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)

# récupère le mot traduit depuis son identifiant
df_tts["tp_phrase_words_tr"] = df_tts["tp_phrase_words_tr_rec_id"].map(
    lambda x:py_modules.set_phrase_words_tr(x, df_tbl_words),
    na_action='ignore')
#=======================================================
# TRADUCTION DES META DONNEES (STORY, THEME, ETC.)
#=======================================================
df_tts["tp_phrase_rel_tp_rec_id"] = df_tts[["tp_phrase_rel_tp_rec_id","tp_phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger
/content/drive/MyDrive/langdeck/assets/audio/ai/dut/recHPZM2bhuSgikRX-dut.mp3(nl) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/dut/rec8WSW27ktZD4Gcw-dut.mp3(nl) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/dut/recAFEMFQvJXhOxJn-dut.mp3(nl) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/eng/recHPZM2bhuSgikRX-eng.mp3(en) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/eng/rec8WSW27ktZD4Gcw-eng.mp3(en) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/eng/recAFEMFQvJXhOxJn-eng.mp3(en) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/fre/recHPZM2bhuSgikRX-fre.mp3(fr) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/fre/rec8WSW27ktZD4Gcw-fre.mp3(fr) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/fre/recAFEMFQvJXhOxJn-fre.mp3(fr

In [ ]:
# récupère le mot traduit depuis son identifiant
def get_tp_tr(rec_id, df):
  vk = []
  for elem in rec_id:
    vk.append(df.loc[df["tp_translation_id"]==elem]["tp_translation"].values.item())
  return vk

In [ ]:
# récupère le mot traduit depuis son identifiant
df_tts["phrase_related_tp_tr"] = df_tts["tp_phrase_rel_tp_rec_id"].map(
    lambda x:get_tp_tr(x, df_tbl_tps),
    na_action='ignore')
# on rebascule les états si il y a eu update
df_tts["tp_phrase_translation_status"] = df_tts[["tp_phrase_translation_status","tp_phrase_status"]].apply(
    lambda x:
    "Pending" if x[1].lower() == "update" else x[0],
    axis=1)
py_modules.save_df_to_gsheet (wb_intgr, "tbl_tp_phrases", df_tts)

# Quiz

### Intégration

In [ ]:
tbd_quiz = Airtable(base_id, "quiz", api_key_airtable)
vw_quiz_grid = tbd_quiz.get_all(
    view='view_grid',
    sort=['quiz_name',],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_quiz = pd.DataFrame.from_records((r['fields'] for r in vw_quiz_grid))
df_quiz.rename(columns={'quiz_rel_tp': 'quiz_rel_tp_rec_id'}, inplace=True)
df_quiz["quiz_rel_tp"] = df_quiz["quiz_rel_tp_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_tps, x, "tp_name"),
    na_action="ignore"
)
df_quiz.rename(columns={'quiz_rel_story': 'quiz_rel_story_rec_id'}, inplace=True)
df_quiz["quiz_rel_story"] = df_quiz["quiz_rel_story_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_stories, x, "story_name"),
    na_action="ignore"
)

# on caste le string en dict
df_quiz["quiz_form_dict"] = df_quiz["quiz_form"].map (lambda x:ast.literal_eval(x))

vk_col_quiz = ["quiz_rec_id",
              "quiz_name",
              "quiz_rel_tp_rec_id",
              "quiz_rel_tp",
              "quiz_rel_story",
              "quiz_rel_story_rec_id",
              "quiz_illustration",
              "quiz_summary",
              "quiz_lesson_header",
              "quiz_form",
              "quiz_form_dict",
              "quiz_form_translation_status",
              ]
df_quiz = df_quiz[vk_col_quiz]

#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_quiz = pd.concat([df_quiz, df_t.reindex(df_quiz.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_quiz,
        id_vars=vk_col_quiz,
        var_name="quiz_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["quiz_translation_id"] = df_w_temp[['quiz_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_quiz = py_modules.load_df_from_gsheet (wb_intgr, "tbl_quiz")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_quiz[["quiz_translation_id",
                  "quiz_translation",
                  "quiz_translation_status",
                  "quiz_audio",
                  "quiz_audio_url",
                  "quiz_audio_url_fr",
                  "quiz_summary_translation",
                  "quiz_summary_translation_status",
                  "quiz_form_translation"
                  ]],
    on="quiz_translation_id",
    how="left")


In [ ]:
df_w_tr['quiz_form_translation']

0    {'quizTitle': 'Bij de dokter', 'quizSynopsis':...
1    {'quizTitle': 'De ontbijtbuffetquiz', 'quizSyn...
2    {'quizTitle': 'At the doctor', 'quizSynopsis':...
3    {'quizTitle': 'The breakfast buffet quiz', 'qu...
4    {'quizTitle': 'Chez le médecin', 'quizSynopsis...
5    {'quizTitle': 'Le quiz du buffet du petit-déje...
6    {'quizTitle': 'Beim Arzt', 'quizSynopsis': 'Me...
7    {'quizTitle': 'Das Frühstücksbuffet-Quiz', 'qu...
Name: quiz_form_translation, dtype: object

In [ ]:
def translate_quiz_form_dict (fr_dict, language):
  tr_dict = {}
  tr_dict['quizTitle'] = py_modules.word_translate_update_status(fr_dict['quizTitle'], "fre", language, "", df_languages)[0]
  tr_dict['quizSynopsis'] = py_modules.word_translate_update_status(fr_dict['quizSynopsis'], "fre", language, "", df_languages)[0]
  tr_dict['questions'] = []
  vk_questions = []
  for q in fr_dict["questions"]:
    tr_dict_q = {}
    tr_dict_q['question'] = py_modules.word_translate_update_status(q['question'], "fre", language, "", df_languages)[0]
    tr_dict_q["questionType"] = q["questionType"]
    vk_answers = []
    for a in q["answers"]:
      vk_answers.append(py_modules.word_translate_update_status(a, "fre", language, "", df_languages)[0])
    tr_dict_q["answers"] = vk_answers
    vk_questions.append (tr_dict_q)
    tr_dict_q["correctAnswer"] = q["correctAnswer"]
  tr_dict['questions'] = vk_questions
  print (tr_dict)
  return tr_dict

In [ ]:
df_w_tr["quiz_form_translation"] = df_w_tr[
    ["quiz_form_dict",
     "quiz_language",]].apply(
      lambda x:translate_quiz_form_dict(x[0],x[1]), axis=1,)

### Traduction

In [ ]:
df_w_tr["quiz_translation"] = df_w_tr["quiz_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["quiz_translation_status"] = df_w_tr["quiz_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["quiz_translation","quiz_translation_status"]] = df_w_tr[["quiz_translation",
                                                                   "quiz_name",
                                                                   "quiz_language",
                                                                   "quiz_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["quiz_audio"] = df_w_tr[["quiz_translation_id","quiz_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

df_w_tr["quiz_summary_translation"] = df_w_tr["quiz_summary_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["quiz_summary_translation_status"] = df_w_tr["quiz_summary_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["quiz_summary_translation","quiz_summary_translation_status"]] = df_w_tr[["quiz_summary_translation",
                                                                   "quiz_summary",
                                                                   "quiz_language",
                                                                   "quiz_summary_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

### TTS

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["quiz_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["quiz_translation","quiz_language","quiz_audio","quiz_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['quiz_audio_url'] = df_tts['quiz_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
df_tts["quiz_audio_url"] = df_tts[["quiz_language","quiz_audio","quiz_audio_url","quiz_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["quiz_audio_url_fr"] = df_tts["quiz_rec_id"].map(
    lambda x:py_modules.table_get_audio_url_fr(x,
                                    df_tts,
                                    "quiz_translation_id",
                                    "quiz_audio_url",
                                    ))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [ ]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_quiz", df_tts)

# Transformation format JSON
Chargement et transformation des tables en dictionnaires JSON

## Langues

In [20]:
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages")
# merge pour récupérer des infos de référentiels
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","country_ref","flag_icon"]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid",], axis=1, inplace=True)
df_languages.rename(
    columns={"country_ref":"lang_country_ref",
             "flag_icon":"lang_flag_icon"},
    inplace=True)
# merge pour récupérer des infos de référentiels
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages_countries")
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","language_countries",]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid","lang_countries_vk",], axis=1, inplace=True)
df_languages.rename(
    columns={"language_countries":"lang_countries_vk",},
    inplace=True)
# Liste des colonnes pour le dict
vk_language_cols = [
    "lang_id",
    "lang_alpha3",
    "lang_alpha2_google",
    "lang_alpha2",
    "lang_wals",
    "lang_name_native",
    "lang_name_en",
    "lang_name_fr",
    "lang_is_available",
    "lang_is_tts",
    "lang_countries_vk",
    "lang_country_ref",
    "lang_flag_icon",
]
vk_lang_vals = df_languages.groupby(["lang_id"])[vk_language_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Themes

In [22]:
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes")
df_themes.sort_values (
    by=["theme_name",
        "theme_language",
        ],
    inplace=True)

vk_themes_cols = [
    "theme_rec_id",
    "theme_name",
    "theme_language",
    "theme_translation",
    "theme_audio",
    "theme_audio_url",
    "theme_audio_url_fr",
    "theme_illustration",
    ]

vk_themes_vals = df_themes.groupby(["theme_rec_id"])[vk_themes_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Lessons

In [23]:
df_lessons = py_modules.load_df_from_gsheet(wb_intgr, "tbl_lessons")
df_lessons.sort_values (
    by=["lesson_number",
        "lesson_language",
        ],
    inplace=True)

df_lessons['lesson_related_theme'] = df_lessons['lesson_related_theme'].map(lambda x:ast.literal_eval(x) if x else [])
df_lessons['lesson_related_theme'] = df_lessons['lesson_related_theme'].map(lambda x:''.join (x))

vk_lessons_cols = [
    "lesson_rec_id",
    "lesson",
    "lesson_language",
    "lesson_story",
    "lesson_pictorial",
    "lesson_quiz",
    "lesson_number",
    "lesson_translation",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr",
    "lesson_illustration",
    "lesson_related_theme",
    ]

vk_lessons_vals = df_lessons.groupby(["lesson_rec_id"])[vk_lessons_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Stories

In [24]:
df_stories = py_modules.load_df_from_gsheet(wb_intgr, "tbl_stories")
df_stories.sort_values (
    by=["story_language",
        "story_level",
        "story_name",
        "story_episode",],
    inplace=True)

df_stories['story_related_lesson'] = df_stories['story_related_lesson'].map(lambda x:ast.literal_eval(x) if x else [])
df_stories['story_related_lesson'] = df_stories['story_related_lesson'].map(lambda x:''.join (x))

vk_stories_cols = [
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_language",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    "story_synopsis",
    "story_synopsis_translation",
    "story_lesson_header",
    "story_related_lesson",
    ]

vk_stories_vals = df_stories.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Phrases

In [25]:
df_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_phrases") # load source sheet
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_rec_id"] = df_phrases["phrase_related_story_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr"] = df_phrases["phrase_related_story_tr"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr_rec_id"] = df_phrases["phrase_related_story_tr_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes") # load source sheet
df_phrases.sort_values(by=['phrase_language','phrase_related_story','phrase_paragraph','phrase_position'], inplace=True)

# merge phrases + story => vue à plat avec les données étendues
df_phr_ext = pd.merge(df_phrases, df_stories[vk_stories_cols],
         left_on="phrase_related_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of columns
vk_phrases_cols = [
    "phrase_language",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_words_tr",
    "phrase_words_tr_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_translation",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_audio_url_fr",
    "phrase_related_story_tr",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_translation",
    "phrase_notes_2_translation",
    ] + vk_stories_cols

    # fields (columns)

# vector of values grouped by language
vk_phrases_vals = df_phr_ext.groupby(["phrase_language","phrase_related_story"])[vk_phrases_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()


## Words

In [26]:
df_words = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words")
df_words["order"] = df_words["word_type"].map(lambda x:1 if x=="Lemma" else 2 if x=="Expression" else 3)
df_words.sort_values(by=['word_language','word','order',], inplace=True)
vk_word_cols = [
    "word_language",
    "word",
    "word_rec_id",
    "order",
    "word_type",
    "word_translation_id",
    "word_translation",
    "word_lemmas",
    "word_lemmas_rec_id",
    "word_audio",
    "word_audio_url",
    "word_audio_url_fr",
    "word_lemmas_tr_rec_id",
    "word_lemmas_tr",
    "word_notes",
    "word_nutri_notes",
    "word_notes_translation",
    "word_nutri_notes_translation",
    ] # fields (columns)

vk_word_vals = df_words.groupby(["word_language"])[vk_word_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## TPs

In [27]:
df_tps = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tps")
df_tps.sort_values (
    by=["tp_name",
        "tp_language",
        ],
    inplace=True)

vk_tps_cols = [
    "tp_rec_id",
    "tp_name",
    "tp_language",
    "tp_rel_story",
    "tp_rel_story_rec_id",
    "tp_illustration",
    "tp_translation",
    "tp_translation_id",
    "tp_audio",
    "tp_audio_url",
    "tp_audio_url_fr",
    "tp_lesson_header",
    "tp_summary",
    "tp_summary_translation",
    ]

#vk_tps_vals = df_tps.groupby(["tp_rec_id"])[vk_tps_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tps["tp_rel_story_rec_id"] = df_tps['tp_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tps["tp_rel_story"] = df_tps['tp_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

df_tps["tp_rel_story_tr_rec_id"] = df_tps[['tp_rel_story_rec_id', 'tp_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

# merge phrases + story => vue à plat avec les données étendues
df_tps_ext = pd.merge(df_tps, df_stories[vk_stories_cols],
         left_on="tp_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_tps_vals = df_tps_ext.groupby(["tp_language","tp_name"])[vk_tps_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Phrases des TPs

In [28]:
df_tp_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tp_phrases")
df_tp_phrases.sort_values (
    by=["tp_phrase_language",
        "tp_phrase_order",
        ],
    inplace=True)

vk_tp_p_cols = [
    "tp_phrase_rec_id",
    "tp_phrase",
    "tp_phrase_html",
    "tp_phrase_html_rec_id",
    "tp_phrase_html_kw",
    "tp_phrase_rel_tp",
    "tp_phrase_rel_tp_rec_id",
    "tp_phrase_order",
    "tp_phrase_language",
    "tp_phrase_translation_id",
    "tp_phrase_translation",
    "tp_phrase_audio",
    "tp_phrase_audio_url",
    "tp_phrase_audio_url_fr",
    "tp_phrase_words_rec_id",
    "tp_phrase_words",
    "tp_phrase_words_tr_rec_id",
    "tp_phrase_words_tr",
    "phrase_related_tp_tr",
    ]

vk_tp_p_vals = df_tp_phrases.groupby(["tp_phrase_rec_id"])[vk_tp_p_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tp_phrases["tp_phrase_rel_tp_rec_id"] = df_tp_phrases['tp_phrase_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tp_phrases["tp_phrase_rel_tp"] = df_tp_phrases['tp_phrase_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

# merge phrases + story => vue à plat avec les données étendues
df_tp_p_ext = pd.merge(df_tp_phrases, df_tps,
         left_on="tp_phrase_rel_tp_rec_id",
         right_on="tp_translation_id",
         how="left")

vk_col_tp_p_ext = vk_tp_p_cols + vk_tps_cols + [
  "tp_phrase_words_tr_rec_id",
  "tp_phrase_words_tr",
  "tp_rel_story_tr_rec_id",
  ]

# vector of values grouped by language
vk_tp_p_vals = df_tp_p_ext.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_col_tp_p_ext].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Quiz

In [29]:
df_quiz = py_modules.load_df_from_gsheet(wb_intgr, "tbl_quiz")
df_quiz.sort_values (
    by=["quiz_name",
        "quiz_language",
        ],
    inplace=True)

vk_quiz_cols = [
    "quiz_rec_id",
    "quiz_name",
    "quiz_language",
    "quiz_rel_tp",
    "quiz_rel_tp_rec_id",
    "quiz_rel_story",
    "quiz_rel_story_rec_id",
    "quiz_illustration",
    "quiz_translation",
    "quiz_translation_id",
    "quiz_audio",
    "quiz_audio_url",
    "quiz_audio_url_fr",
    "quiz_summary_translation",
    "quiz_form",
    "quiz_form_translation",
    "quiz_lesson_header",
    ] + vk_stories_cols

df_quiz["quiz_rel_tp_rec_id"] = df_quiz['quiz_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_tp"] = df_quiz['quiz_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story_rec_id"] = df_quiz['quiz_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story"] = df_quiz['quiz_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_form"] = df_quiz["quiz_form"].map (lambda x:ast.literal_eval(x))
df_quiz["quiz_form_translation"] = df_quiz["quiz_form_translation"].map (lambda x:ast.literal_eval(x))

df_quiz["quiz_rel_tp_tr_rec_id"] = df_quiz[['quiz_rel_tp_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
df_quiz["quiz_rel_story_tr_rec_id"] = df_quiz[['quiz_rel_story_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

df_quiz_ext = pd.merge(df_quiz, df_stories[vk_stories_cols],
         left_on="quiz_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_quiz_vals = df_quiz_ext.groupby(["quiz_language"])[vk_quiz_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Fonctions de remplissage au format JSON

In [32]:
def add_dict_of_words (df_words,vk_word_cols, filter_language, vk_filter_words):
  vk_dict_story_phrase_words = []    # on crée un vecteur (liste) pour recevoir les mots de la phrase
  # on filtre sur langue ET identifiants des mots
  # print (len(vk_filter_words))
  if vk_filter_words :
    df_story_phrase_words_filtered = df_words.loc[
        (df_words['word_language']==filter_language) &
        (df_words['word_rec_id'].isin(vk_filter_words))
        ]
      # on créé un vecteur de tuples sur cette liste filtrée
    vk_story_phrase_words = df_story_phrase_words_filtered.groupby(["word_language",])[vk_word_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    # on itère le vecteur de mots => transforme le vecteur de tuples en liste de dicts
    for vk_w in vk_story_phrase_words:
      # print (f'phrase={vk_w}')
      vk_w_dict = list(map(lambda x: dict(zip(vk_word_cols,x)), vk_w))
      # on affecte chaque k/v au dict de destination d_word
      for vk_w_kv in vk_w_dict:
        d_word = {}
        # clé kv
        d_word["word_rec_id"] = vk_w_kv['word_rec_id']
        d_word["word"] = vk_w_kv['word']
        d_word["word_translation"] = vk_w_kv['word_translation']
        vk_dict_story_phrase_words.append(d_word)
  return vk_dict_story_phrase_words


In [33]:
def add_dict_of_story_phrases(df_phr_ext, vk_phrases_cols, filter_language, filter_story):
  debug = False
  #print (f'{df_phr_ext} {vk_phrases_cols} filter_language={filter_language} filter_story={filter_story}')
  vk_dict_story_phrases = []    # on crée un vecteur (liste) pour recevoir les phrases
  # on filtre sur langue ET story
  df_story_phrases_filtered = df_phr_ext.loc[(df_phr_ext['phrase_language']==filter_language) & (df_phr_ext['phrase_related_story']==filter_story)]
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_story_phrases_vals = df_story_phrases_filtered.groupby(["phrase_language","phrase_related_story"])[vk_phrases_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de phrases => transforme le vecteur de tuples en liste de dicts
  for vk_phr in vk_story_phrases_vals:
    # print (f'phrase={vk_phr}')
    vk_phr_dict = list(map(lambda x: dict(zip(vk_phrases_cols,x)), vk_phr))
    # on affecte chaque k/v au dict de destination d_phrase
    for vk_phr_kv in vk_phr_dict:
      d_phrase = {}
      # clé kv
      d_phrase["phrase_rec_id"] = vk_phr_kv['phrase_rec_id']
      if not debug:
        # on ajoute les autres colonnes
        for vkcol in vk_phrases_cols:
          d_phrase[vkcol] = vk_phr_kv[vkcol]
      ## on ajoute les mots
      list_words = ast.literal_eval(d_phrase["phrase_words_rec_id"])
      d_phrase["words"] = add_dict_of_words (df_words, vk_word_cols, filter_language, list_words)
      vk_dict_story_phrases.append(d_phrase)

  return vk_dict_story_phrases

In [34]:
def add_dict_of_tp_phrases (df_tp_p_ext, vk_col_tp_p_ext, filter_language, filter_story):
  vk_dict_of_tp_phrases = []    # on crée un vecteur (liste) pour recevoir les phrases
  # on filtre sur langue ET story
  df_tp_phrases_filtered = df_tp_p_ext.loc[(df_tp_p_ext['tp_phrase_language']==filter_language) & (df_tp_p_ext['tp_phrase_rel_tp']==filter_story)]
  # si le vecteur de mots est vide, liste vide plutôt que NaN
  df_tp_phrases_filtered['tp_phrase_words_rec_id'] = df_tp_phrases_filtered['tp_phrase_words_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_tp_phrases_vals = df_tp_phrases_filtered.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_col_tp_p_ext].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de phrases => transforme le vecteur de tuples en liste de dicts
  for vk_phr in vk_tp_phrases_vals:
    print (f'phrase={vk_phr}')
    vk_phr_dict = list(map(lambda x: dict(zip(vk_col_tp_p_ext,x)), vk_phr))
    # on affecte chaque k/v au dict de destination d_phrase
    for vk_phr_kv in vk_phr_dict:
      d_phrase = {}
      # clé kv
      d_phrase["tp_phrase_rec_id"] = vk_phr_kv['tp_phrase_rec_id']
      d_phrase["tp_phrase_words_rec_id"] = vk_phr_kv['tp_phrase_words_rec_id']
      ## on ajoute les mots
      list_words = d_phrase['tp_phrase_words_rec_id']
      d_phrase["words"] = add_dict_of_words (df_words, vk_word_cols, filter_language, list_words)
      vk_dict_of_tp_phrases.append(d_phrase)
  return vk_dict_of_tp_phrases

In [35]:
def add_dict_of_story_quiz (df_quiz_ext, vk_quiz_cols, filter_language, filter_story):
  vk_dict_story_quiz = []    # on crée un vecteur (liste) pour recevoir les phrases
  df_story_quiz_filtered = df_quiz_ext.loc[(df_quiz_ext['quiz_language']==filter_language) & (df_quiz_ext['quiz_rel_story']==filter_story)]
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_story_quiz_vals = df_story_quiz_filtered.groupby(["quiz_language","quiz_name"])[vk_quiz_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de tp (singleton !) => transforme le vecteur de tuples en liste de dicts
  for vk_quiz in vk_story_quiz_vals:
    vk_quiz_dict = list(map(lambda x: dict(zip(vk_quiz_cols,x)), vk_quiz))
    for vk_quiz_kv in vk_quiz_dict:
      d_quiz = {}
      d_quiz['quiz_name'] = vk_quiz_kv['quiz_name']
      d_quiz['quiz_translation'] = vk_quiz_kv['quiz_translation']

      # quiz form
      d_quiz['quiz_form'] = vk_quiz_kv['quiz_form']

      # quiz form translation
      d_quiz['quiz_form_translation'] = vk_quiz_kv['quiz_form_translation']

      vk_dict_story_quiz.append(d_quiz)

  return vk_dict_story_quiz

In [36]:
def add_dict_of_story_tp (df_tps_ext, vk_tps_cols, filter_language, filter_story):
  vk_dict_story_tp = []    # on crée un vecteur (liste) pour recevoir les phrases
  df_story_tp_filtered = df_tps_ext.loc[(df_tps_ext['tp_language']==filter_language) & (df_tps_ext['tp_rel_story']==filter_story)]
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_story_tp_vals = df_story_tp_filtered.groupby(["tp_language","tp_name"])[vk_tps_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de tp (singleton !) => transforme le vecteur de tuples en liste de dicts
  for vk_tp in vk_story_tp_vals:
    vk_tp_dict = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk_tp))
    for vk_tp_kv in vk_tp_dict:
      d_tp = {}
      d_tp['tp_name'] = vk_tp_kv['tp_name']
      d_tp['tp_translation'] = vk_tp_kv['tp_translation']
      d_tp['tp_phrases'] = add_dict_of_tp_phrases(
          df_tp_p_ext,
          vk_col_tp_p_ext,
          filter_language,
          d_tp['tp_name']
          )
      vk_dict_story_tp.append(d_tp)

  return vk_dict_story_tp

## Niveau Stories

In [37]:
def add_dict_of_stories (df_stories, vk_stories_cols, filter_language, filter_lesson):
  debug = True
  vk_dict_stories = []

  df_stories_filtered = df_stories.loc[(df_stories['story_language']==filter_language) & (df_stories['story_related_lesson']==filter_lesson)]
  vk_stories_vals = df_stories_filtered.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  for vk_s in vk_stories_vals:
    vk_s_dict = list(map(lambda x: dict(zip(vk_stories_cols,x)), vk_s))
    d_story = {}
    d_story["story"] = list(dict.fromkeys(map(
      lambda x:x["story_name"],
      sorted(vk_s_dict, key=lambda x: x["story_rec_id"]))))[0]
    if not debug:
      ## colonnes story
      for vkcol in vk_stories_cols:
        d_story[vkcol] = list(dict.fromkeys(map(
          lambda x:x[vkcol],
          sorted(vk_s_dict, key=lambda x: x["story_rec_id"]))))[0]
    ## on ajoute les phrases
    d_story['phrases'] = add_dict_of_story_phrases (
        df_phr_ext,
        vk_phrases_cols,
        filter_language,
        d_story["story"]
        )
    ## on ajoute le TP
    d_story['tps'] = add_dict_of_story_tp (
        df_tps_ext,
        vk_tps_cols,
        filter_language,
        d_story["story"]
        )
    ## on ajoute le Quiz
    d_story['quiz'] = add_dict_of_story_quiz (
        df_quiz_ext,
        vk_quiz_cols,
        filter_language,
        d_story["story"]
        )
    vk_dict_stories.append(d_story)
  return vk_dict_stories


## Niveau Lesson

In [38]:
def add_dict_of_lessons (df, vk_cols, filter_language, filter_theme):
  debug = True
  print (f'>> add_dict_of_lessons : {filter_language} {filter_theme}')
  vk_dict_returned = []
  df_filtered = df.loc[(df['lesson_language']==filter_language) & (df['lesson_related_theme']==filter_theme)]
  # si thème possède des lessons
  if len(df_filtered) > 0:
    vk_vals = df_filtered.groupby(["lesson_rec_id"])[vk_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    for vk in vk_vals:
      vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
      d_lesson = {}
      d_lesson["lesson"] = list(dict.fromkeys(map(
        lambda x:x["lesson"],
        sorted(vk_dict, key=lambda x: x["lesson_rec_id"]))))[0]
      d_lesson["language"] = filter_language
      d_lesson["lesson_rec_id"] = list(dict.fromkeys(map(
        lambda x:x["lesson_rec_id"],
        sorted(vk_dict, key=lambda x: x["lesson_rec_id"]))))[0]

      # on ajoute les stories
      d_lesson['stories'] = add_dict_of_stories (
        df_stories,
        vk_stories_cols,
        d_lesson['language'],
        d_lesson["lesson_rec_id"])

    vk_dict_returned.append(d_lesson)

  return vk_dict_returned


## Niveau Thème

In [52]:
def add_dict_of_themes (df, vk_cols, filter_language):
  print (f'>> add_dict_of_themes : {filter_language}')
  vk_dict_returned = []
  df_filtered = df.loc[df['theme_language']==filter_language]
  vk_vals = df_filtered.groupby(["theme_rec_id"])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  for vk in vk_vals:
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    d_key = {}

    ## colonnes suppl.
    for vkcol in vk_cols:
      d_key[vkcol] = list(dict.fromkeys(map(
        lambda x:x[vkcol],
        sorted(vk_dict, key=lambda x: x["theme_rec_id"]))))[0]

    # on ajoute les lessons
    d_key['lessons'] = add_dict_of_lessons (
        df_lessons,
        vk_lessons_cols,
        d_key['theme_language'],
        d_key["theme_rec_id"]
        )

    vk_dict_returned.append(d_key)

  return vk_dict_returned


In [53]:
vk_langdeck = []
### languages
for vk_l in vk_lang_vals:
  vk_l_dict = list(map(lambda x: dict(zip(vk_language_cols,x)), vk_l))
  d_lang = {}
  d_lang["language"] = list(dict.fromkeys(map(
      lambda x:x["lang_id"],
      sorted(vk_l_dict, key=lambda x: x["lang_id"]))))[0]

  ### stories
  #d_lang['stories']=[]
  d_lang['themes'] = add_dict_of_themes (
      df_themes,
      vk_themes_cols,
      d_lang['language'])
  vk_langdeck.append (d_lang)
  """
  d_lang['stories'] = add_dict_of_stories (
      df_stories,
      vk_stories_cols,
      d_lang['language'])
  vk_langdeck.append (d_lang)
  """

>> add_dict_of_themes : dut
>> add_dict_of_lessons : dut recXrgjiwJhADauYJ
phrase=[('recY1tS2A6OPdUYvP', 'Savourez un copieux petit-déjeuner pendant votre séjour à Tours', 'Savourez un copieux <span id="recrYaJMdw31vgPT5">petit-déjeuner</span> pendant votre séjour à Tours', 'Savourez un copieux <kw>recrYaJMdw31vgPT5<kw> pendant votre séjour à Tours', 'Savourez un copieux <kw>petit-déjeuner<kw> pendant votre séjour à Tours', 'Le buffet', 'recolhmJ3G9UQuKPi-dut', '1', 'dut', 'recY1tS2A6OPdUYvP-dut', 'Geniet van een stevig ontbijt tijdens uw verblijf in Tours', 'assets/audio/ai/dut/recY1tS2A6OPdUYvP-dut.mp3', 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1701709092/langdeck/assets/audio/ai/dut/recY1tS2A6OPdUYvP-dut.mp3', 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1701709101/langdeck/assets/audio/ai/fre/recY1tS2A6OPdUYvP-fre.mp3', ['recrYaJMdw31vgPT5'], "['Petit-déjeuner']", "['recrYaJMdw31vgPT5-dut']", "['Ontbijt']", "['Het buffet']", 'recolhmJ3G9UQuKPi', 'Le buffet', 'dut', 

Ensuite, on itère sur les stories filtrées sur une langue

In [51]:
vk_langdeck[3]

{'language': 'ger',
 'themes': [{'theme_rec_id': 'recXrgjiwJhADauYJ',
   'theme_name': 'Alimentation',
   'theme_language': 'ger',
   'theme_translation': 'Essen',
   'theme_audio': 'assets/audio/ai/ger/recXrgjiwJhADauYJ-ger.mp3',
   'theme_audio_url': 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1697032749/langdeck/assets/audio/ai/ger/recXrgjiwJhADauYJ-ger.mp3',
   'theme_audio_url_fr': 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1697032747/langdeck/assets/audio/ai/fre/recXrgjiwJhADauYJ-fre.mp3',
   'theme_illustration': 'https://res.cloudinary.com/dhc7ovnwk/image/upload/v1704534134/langdeck/assets/images/themes/sfjtgfprbbnepf3ty1mw.jpg',
   'lessons': [{'lesson': 'Le petit-déjeuner',
     'language': 'ger',
     'lesson_rec_id': 'recXmlJ9UxenIi1gr',
     'stories': [{'story': 'Le petit-déjeuner',
       'phrases': [{'phrase_rec_id': 'recUTxQtXiufCX1Nt',
         'phrase_language': 'ger',
         'phrase_related_story': 'Le petit-déjeuner',
         'phrase_related_story

In [ ]:
with open('themesdeck.txt', 'w') as f:
  json.dump(vk_langdeck, f, ensure_ascii=False)

In [ ]:
def get_tps (language, story, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_language"] == language and vk_temp[0]["tp_rel_story"] == story:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict

In [ ]:
for vk_s in vk_stories_vals:
  vk_s_dict = list(map(lambda x: dict(zip(vk_stories_cols,x)), vk_s))
  d_story = {}
  d_story["story"] = list(dict.fromkeys(map(
    lambda x:x["story_name"],
    sorted(vk_s_dict, key=lambda x: x["story_name"]))))[0]
  print (d_story)

{'story': 'Chez le médecin'}
{'story': 'Le petit-déjeuner'}


## Themes

In [ ]:
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes")
df_themes.sort_values (
    by=["theme_name",
        "theme_language",
        ],
    inplace=True)

vk_themes_cols = [
    "theme_rec_id",
    "theme_name",
    "theme_language",
    "theme_translation",
    "theme_audio",
    "theme_audio_url",
    "theme_audio_url_fr",
    ]

vk_themes_vals = df_themes.groupby(["theme_rec_id"])[vk_themes_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Lessons

In [ ]:
df_lessons = py_modules.load_df_from_gsheet(wb_intgr, "tbl_lessons")
df_lessons.sort_values (
    by=["lesson_number",
        "lesson_language",
        ],
    inplace=True)

vk_lessons_cols = [
    "lesson_rec_id",
    "lesson",
    "lesson_language",
    "lesson_story",
    "lesson_pictorial",
    "lesson_quiz",
    "lesson_number",
    "lesson_translation",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr",
    "lesson_illustration",
    "lesson_related_theme",
    ]

vk_lessons_vals = df_lessons.groupby(["lesson_rec_id"])[vk_lessons_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Stories

In [ ]:
df_stories = py_modules.load_df_from_gsheet(wb_intgr, "tbl_stories")
df_stories.sort_values (
    by=["story_language",
        "story_level",
        "story_name",
        "story_episode",],
    inplace=True)
vk_stories_cols = [
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    "story_synopsis",
    "story_synopsis_translation",
    "story_lesson_header",
    "story_related_lesson",
    ]

vk_stories_vals = df_stories.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## TPs

In [ ]:
def get_tps (language, story, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_language"] == language and vk_temp[0]["tp_rel_story"] == story:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict

df_tps = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tps")
df_tps.sort_values (
    by=["tp_name",
        "tp_language",
        ],
    inplace=True)

vk_tps_cols = [
    "tp_rec_id",
    "tp_name",
    "tp_language",
    "tp_rel_story",
    "tp_rel_story_rec_id",
    "tp_illustration",
    "tp_translation",
    "tp_translation_id",
    "tp_audio",
    "tp_audio_url",
    "tp_audio_url_fr",
    "tp_lesson_header",
    "tp_summary",
    "tp_summary_translation",
    ]

#vk_tps_vals = df_tps.groupby(["tp_rec_id"])[vk_tps_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tps["tp_rel_story_rec_id"] = df_tps['tp_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tps["tp_rel_story"] = df_tps['tp_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

df_tps["tp_rel_story_tr_rec_id"] = df_tps[['tp_rel_story_rec_id', 'tp_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

# merge phrases + story => vue à plat avec les données étendues
df_tps_ext = pd.merge(df_tps, df_stories[vk_stories_cols],
         left_on="tp_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_tps_vals = df_tps_ext.groupby(["tp_language","tp_name"])[vk_tps_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

### Phrases des TPs

In [ ]:
def get_tp_phrases (language, tp, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_phrase_language"] == language and vk_temp[0]["tp_rec_id"] == tp:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict


df_tp_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tp_phrases")
df_tp_phrases.sort_values (
    by=["tp_phrase_language",
        "tp_phrase_order",
        ],
    inplace=True)

vk_tp_p_cols = [
    "tp_phrase_rec_id",
    "tp_phrase",
    "tp_phrase_html",
    "tp_phrase_html_rec_id",
    "tp_phrase_html_kw",
    "tp_phrase_rel_tp",
    "tp_phrase_rel_tp_rec_id",
    "tp_phrase_order",
    "tp_phrase_language",
    "tp_phrase_translation_id",
    "tp_phrase_translation",
    "tp_phrase_audio",
    "tp_phrase_audio_url",
    "tp_phrase_audio_url_fr",
    "tp_phrase_words_rec_id",
    "tp_phrase_words",
    "tp_phrase_words_tr_rec_id",
    "tp_phrase_words_tr",
    "phrase_related_tp_tr",
    ]

vk_tp_p_vals = df_tp_phrases.groupby(["tp_phrase_rec_id"])[vk_tp_p_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tp_phrases["tp_phrase_rel_tp_rec_id"] = df_tp_phrases['tp_phrase_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tp_phrases["tp_phrase_rel_tp"] = df_tp_phrases['tp_phrase_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

# merge phrases + story => vue à plat avec les données étendues
df_tp_p_ext = pd.merge(df_tp_phrases, df_tps,
         left_on="tp_phrase_rel_tp_rec_id",
         right_on="tp_translation_id",
         how="left")

vk_col_tp_p_ext = vk_tp_p_cols + vk_tps_cols + [
  "tp_phrase_words_tr_rec_id",
  "tp_phrase_words_tr",
  "tp_rel_story_tr_rec_id",
  ]

# vector of values grouped by language
vk_tp_p_vals = df_tp_p_ext.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_col_tp_p_ext].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

#### Mots des phrases des TPs

In [ ]:
def get_tp_phrase_words (vk_tp_phrases):
  for tp_phrase in vk_tp_phrases:
    print (tp_phrase["tp_phrase_words_tr_rec_id"])
    vk_words = []
    if len(tp_phrase["tp_phrase_words_tr_rec_id"])>0:
      list_words = ast.literal_eval(tp_phrase["tp_phrase_words_tr_rec_id"])
      for word in list_words:
        #------W O R D S----------
        d_word = {}
        # on récupère la valeur pour chaque attribut de mot
        for field in vk_word_cols:
          d_word[field] = f_words_get_value_by_col(
              df_words,
              "word_translation_id",
              field,
              word)

        d_word["word_tr_rec_id"] = word
        vk_words.append(d_word)
    tp_phrase["words"] = vk_words

  return vk_tp_phrases

"""
vk_tp_phrases = get_tp_phrases(
    d_lang["language"],
    vk_tps[0]["tp_rec_id"],
    vk_tp_p_vals,
    vk_col_tp_p_ext,
    )
get_tp_phrase_words (vk_tp_phrases)
"""

'\nvk_tp_phrases = get_tp_phrases(\n    d_lang["language"],\n    vk_tps[0]["tp_rec_id"],\n    vk_tp_p_vals,\n    vk_col_tp_p_ext,\n    )\nget_tp_phrase_words (vk_tp_phrases)\n'

## Quiz

In [ ]:
def get_quiz (language, tp, vk_tuple_quiz, vk_quiz_cols):
  vk_lang_quiz_dict = []
  for vk in vk_tuple_quiz:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_quiz_cols,x)), vk))
    if vk_temp[0]["quiz_language"] == language and vk_temp[0]["quiz_rel_tp"] == tp:
      vk_lang_quiz_dict = vk_temp
  return vk_lang_quiz_dict

def get_quiz_by_story (language, story, vk_tuple_quiz, vk_quiz_cols):
  vk_lang_quiz_dict = []
  for vk in vk_tuple_quiz:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_quiz_cols,x)), vk))
    if vk_temp[0]["quiz_language"] == language and vk_temp[0]["quiz_rel_story"] == story:
      vk_lang_quiz_dict = vk_temp
  return vk_lang_quiz_dict

In [ ]:
df_quiz = py_modules.load_df_from_gsheet(wb_intgr, "tbl_quiz")
df_quiz.sort_values (
    by=["quiz_name",
        "quiz_language",
        ],
    inplace=True)

In [ ]:
vk_quiz_cols = [
    "quiz_rec_id",
    "quiz_name",
    "quiz_language",
    "quiz_rel_tp",
    "quiz_rel_tp_rec_id",
    "quiz_rel_story",
    "quiz_rel_story_rec_id",
    "quiz_illustration",
    "quiz_translation",
    "quiz_translation_id",
    "quiz_audio",
    "quiz_audio_url",
    "quiz_audio_url_fr",
    "quiz_summary_translation",
    "quiz_form",
    "quiz_form_translation",
    "quiz_lesson_header",
    ] + vk_stories_cols


In [ ]:
#vk_quiz_vals = df_quiz.groupby(["quiz_rec_id"])[vk_quiz_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_quiz["quiz_rel_tp_rec_id"] = df_quiz['quiz_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_tp"] = df_quiz['quiz_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story_rec_id"] = df_quiz['quiz_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story"] = df_quiz['quiz_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_form"] = df_quiz["quiz_form"].map (lambda x:ast.literal_eval(x))
df_quiz["quiz_form_translation"] = df_quiz["quiz_form_translation"].map (lambda x:ast.literal_eval(x))

df_quiz["quiz_rel_tp_tr_rec_id"] = df_quiz[['quiz_rel_tp_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
df_quiz["quiz_rel_story_tr_rec_id"] = df_quiz[['quiz_rel_story_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

In [ ]:
# merge phrases + story => vue à plat avec les données étendues
df_quiz_ext = pd.merge(df_quiz, df_stories[vk_stories_cols],
         left_on="quiz_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")


In [ ]:
# vector of values grouped by language
vk_quiz_vals = df_quiz_ext.groupby(["quiz_language"])[vk_quiz_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Words and keywords

In [ ]:
df_words = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words")
df_words["order"] = df_words["word_type"].map(lambda x:1 if x=="Lemma" else 2 if x=="Expression" else 3)
df_words.sort_values(by=['word_language','word','order',], inplace=True)
vk_word_cols = [
    "word_language",
    "word",
    "word_rec_id",
    "order",
    "word_type",
    "word_translation_id",
    "word_translation",
    "word_lemmas",
    "word_lemmas_rec_id",
    "word_audio",
    "word_audio_url",
    "word_audio_url_fr",
    "word_lemmas_tr_rec_id",
    "word_lemmas_tr",
    "word_notes",
    "word_nutri_notes",
    "word_notes_translation",
    "word_nutri_notes_translation",
    ] # fields (columns)

vk_word_vals = df_words.groupby(["word_language"])[vk_word_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Keywords

In [ ]:
def f_phrases_list_to_dict_by_col(col_name, vkp):
  vkf = list(dict.fromkeys(list(map(
        lambda x: x[col_name],
        filter(lambda d: d['phrase_rec_id'] in vkp,
               sorted(vk_lang_dict, key=lambda x: x["phrase_paragraph"]))))))[0]
  return vkf

def f_words_get_value_by_col (df, col_key, col_name, key):
  cval = df.loc[df[col_key]==key][col_name].values.item()
  return cval


## Languages

In [ ]:
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages")
# merge pour récupérer des infos de référentiels
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","country_ref","flag_icon"]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid",], axis=1, inplace=True)
df_languages.rename(
    columns={"country_ref":"lang_country_ref",
             "flag_icon":"lang_flag_icon"},
    inplace=True)
# merge pour récupérer des infos de référentiels
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages_countries")
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","language_countries",]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid","lang_countries_vk",], axis=1, inplace=True)
df_languages.rename(
    columns={"language_countries":"lang_countries_vk",},
    inplace=True)
# Liste des colonnes pour le dict
vk_language_cols = [
    "lang_id",
    "lang_alpha3",
    "lang_alpha2_google",
    "lang_alpha2",
    "lang_wals",
    "lang_name_native",
    "lang_name_en",
    "lang_name_fr",
    "lang_is_available",
    "lang_is_tts",
    "lang_countries_vk",
    "lang_country_ref",
    "lang_flag_icon",
]
vk_lang_vals = df_languages.groupby(["lang_id"])[vk_language_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

{'story': 'Chez le médecin'}
{'story': 'Le petit-déjeuner'}


In [ ]:
vk_lang = []
# pour chaque phrase : on itère
for vk_lang_tuple in vk_phrases_vals:
  # on transforme le tuple en dict
  vk_lang_dict = list(map(lambda x: dict(zip(vk_phrases_cols,x)), vk_lang_tuple))
  d_lang = {}
  #------L A N G U A G E S----------
  d_lang["language"] = list(dict.fromkeys(map(
      lambda x:x["phrase_language"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_language"]))))[0]
  # get all language cols by key
  for field in vk_language_cols:
    d_lang[field] = df_languages.loc[df_languages["lang_id"]==d_lang["language"]][field].values.item()
  # ce vecteur est dans un format string, il faut le caster en array
  d_lang["lang_countries_vk"] = ast.literal_eval(d_lang["lang_countries_vk"])
  # pour chaque story
  vk_stories = []
  for el_story in list(dict.fromkeys(list(map(
      lambda x: x["phrase_related_story"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_related_story"]))))):
    d_story = {}
    #------S T O R I E S----------
    d_story["story"] = el_story

    #------ T P S----------
    # pour chaque TP
    vk_tps = get_tps(d_lang["language"],
                     el_story,
                     vk_tps_vals,
                     vk_tps_cols,
                     )



    # on ajoute les phrases associées
    vk_tps[0]["tp_phrases"] = get_tp_phrases(
        d_lang["language"],
        vk_tps[0]["tp_rec_id"],
        vk_tp_p_vals,
        vk_col_tp_p_ext,
        )
    # on ajoute les mots récursivement
    vk_tp_phrases = get_tp_phrases(
      d_lang["language"],
      vk_tps[0]["tp_rec_id"],
      vk_tp_p_vals,
      vk_col_tp_p_ext,
      )

    vk_tps = get_tp_phrase_words (vk_tp_phrases)


    d_story["tp_name"] = vk_tps[0]["tp_name"]
    d_story["tp_illustration"] = vk_tps[0]["tp_illustration"]
    d_story["tp_language"] = vk_tps[0]["tp_language"]
    d_story["tp_translation"] = vk_tps[0]["tp_translation"]
    d_story["tp_audio"] = vk_tps[0]["tp_audio"]
    d_story["tp_audio_url"] = vk_tps[0]["tp_audio_url"]
    d_story["tp_audio_url_fr"] = vk_tps[0]["tp_audio_url_fr"]
    d_story["tp_lesson_header"] = vk_tps[0]["tp_lesson_header"]
    d_story["tp_summary"] = vk_tps[0]["tp_summary"]
    d_story["tp_summary_translation"] = vk_tps[0]["tp_summary_translation"]

    d_story["tps"] = vk_tps
    d_story["tp_quiz"] = get_quiz_by_story(d_lang["language"],
              d_story["story"],
              vk_quiz_vals,
              vk_quiz_cols,
              )
    # d_story["tps"]["tp_phrases"] = vk_tps
    # pour chaque phrase
    vk_phrases = []
    for phrase_rec_id in list(dict.fromkeys(list(map(
        lambda x: x["phrase_rec_id"],
        filter(lambda d: d['phrase_related_story'] == el_story,
               sorted(vk_lang_dict, key=lambda x: int(x["phrase_paragraph"]))))))):
      d_phrase = {}
      #------P H R A S E S----------
      # pour chaque colonne, on tranforme en dict
      for field in vk_phrases_cols:
        d_phrase[field] = f_phrases_list_to_dict_by_col(
            field,
            phrase_rec_id)
      d_phrase["phrase_rec_id"] = phrase_rec_id
      # rendons à story ce qui appartient à story...
      for field in vk_stories_cols:
        d_story[field] = d_phrase[field]
        d_phrase.pop(field)

      # pour chaque mot
      vk_words = []
      if len(d_phrase["phrase_words_tr_rec_id"])>0:
        list_words = ast.literal_eval(d_phrase["phrase_words_tr_rec_id"])
        for word in list_words:
          #------W O R D S----------
          d_word = {}
          # on récupère la valeur pour chaque attribut de mot
          for field in vk_word_cols:
            d_word[field] = f_words_get_value_by_col(
                df_words,
                "word_translation_id",
                field,
                word)

          d_word["word_tr_rec_id"] = word
          # lemmes et expressions
          vk_lem_exp = []
          # si lemmes : on itère le vecteur
          if len(d_word["word_lemmas_rec_id"])>0:
            list_lems = ast.literal_eval(d_word["word_lemmas_rec_id"])
            for lem in list_lems:
              #---K E Y W O R D S---
              d_lem = {}
              # on construit la clé de recherche id + langue
              lemkey = lem + "-" + d_word["word_language"]
              # pour chaque colonne (idem words) : on récupère la valeur à partir de la clé
              for field in vk_word_cols:
                d_lem[field] = f_words_get_value_by_col(
                  df_words,
                  "word_translation_id",
                  field,
                  lemkey)
              #----------------------
              d_lem["keyword"] = lem
              vk_lem_exp.append(d_lem)
              #----------------------

          #----------------------
          d_word["keywords"] = vk_lem_exp
          vk_words.append(d_word)
          #----------------------

      #----------------------
      d_phrase["words"] = vk_words
      vk_phrases.append(d_phrase)
      #----------------------

    #----------------------
    d_story["phrases"] = vk_phrases
    #----------------------
    vk_stories.append(d_story)


  #----------------------
  d_lang["stories"] = vk_stories
  #----------------------

  vk_lang.append(d_lang)


In [ ]:
len(vk_stories)

1

In [ ]:
vk_lang[0]

In [ ]:
vk_tp_phrases

[]

In [ ]:
vk_lang[3]['stories']

[{'story': 'Le petit-déjeuner',
  'tp_name': 'Le buffet',
  'tp_illustration': 'https://res.cloudinary.com/dhc7ovnwk/image/upload/v1702561868/langdeck/assets/images/saynetes/%C3%A9tude-bleu.png',
  'tp_language': 'eng',
  'tp_translation': 'The buffet',
  'tp_audio': 'assets/audio/ai/eng/recolhmJ3G9UQuKPi-eng.mp3',
  'tp_audio_url': 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1701706854/langdeck/assets/audio/ai/eng/recolhmJ3G9UQuKPi-eng.mp3',
  'tp_audio_url_fr': 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1701706854/langdeck/assets/audio/ai/fre/recolhmJ3G9UQuKPi-fre.mp3',
  'tp_lesson_header': 'https://res.cloudinary.com/dhc7ovnwk/image/upload/v1702660486/langdeck/assets/images/saynetes/langdeck-lesson-header-2.png',
  'tp_summary': "Le texte à étudier est extrait du site web d'un hôtel qui propose un buffet pour le petit-déjeuner. Le texte contient beaucoup de vocabulaire. Etudiez le menu phrase par phrase et essayez de retenir les mots.",
  'tp_summary_translation': 'T

## Sauvegarde au format JSON

In [ ]:
with open('saynetes.txt', 'w') as f:
  json.dump(vk_lang, f, ensure_ascii=False)